Note : Used google collab for this assignment.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import itertools
from itertools import chain
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [ ]:

#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')

In [ ]:
#pip install textstat

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}


# Data Extraction

In [ ]:
df = pd.read_csv("/content/Input.csv")
df = pd.DataFrame(df)
df

,URL_ID,URL
0,1,https://insights.blackcoffer.com/how-is-login-...
1,2,https://insights.blackcoffer.com/how-does-ai-h...
2,3,https://insights.blackcoffer.com/ai-and-its-im...
3,4,https://insights.blackcoffer.com/how-do-deep-l...
4,5,https://insights.blackcoffer.com/how-artificia...
...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...
166,168,https://insights.blackcoffer.com/sales-forecas...
167,169,https://insights.blackcoffer.com/detect-data-e...
168,170,https://insights.blackcoffer.com/data-exfiltra...


In [ ]:
df['URL']

0      https://insights.blackcoffer.com/how-is-login-...
1      https://insights.blackcoffer.com/how-does-ai-h...
2      https://insights.blackcoffer.com/ai-and-its-im...
3      https://insights.blackcoffer.com/how-do-deep-l...
4      https://insights.blackcoffer.com/how-artificia...
                             ...                        
165    https://insights.blackcoffer.com/role-big-data...
166    https://insights.blackcoffer.com/sales-forecas...
167    https://insights.blackcoffer.com/detect-data-e...
168    https://insights.blackcoffer.com/data-exfiltra...
169    https://insights.blackcoffer.com/impacts-of-co...
Name: URL, Length: 170, dtype: object

In [ ]:
#function to scrape content and save in a file.

def get_title(url):
  r = requests.get(url, headers = headers)
  soup = BeautifulSoup(r.text, 'html.parser')
  art_title = soup.find( 'span',{ 'class' : "td-bred-no-url-last"}).text
  art_content  = soup.find( 'div',{ 'class' : "td-post-content"}).text

  f = open("170.txt", "w")
  f.writelines(art_title)
  f.writelines(art_content)
  f.close()

  
get_title('https://insights.blackcoffer.com/impacts-of-covid-19-on-vegetable-vendors-and-food-stalls/')


In [ ]:
# removing last line of the txt file (Author name)

with open("/content/170.txt","r+") as f:
    text = f.readlines()
    f.seek(0)
    f.truncate()
    f.writelines(text[:-1])
    f.close()

# 1. Sentiment Analysis

In [ ]:
# Remove stop words from txt file

In [ ]:
k = []
z = []

with open('/content/stopWords.txt', 'r') as x:
    for l in x:
        for word in l.split():
           k.append(word)

# tokenizing words without using NLTK

#with open('2.txt', 'r') as f:
#    for line in f:
#        for word in line.split():
#           z.append(word)



# tokenzing words using NLTK

with open('170.txt', 'r') as f:
  for lines in f:  
    line = f.read()
    #print(lines)
    z = word_tokenize(line)

    
p = [t for t in z if t.upper() not in k]


In [ ]:
master_df = pd.read_csv("/content/masterDict.csv")

In [ ]:
# list of negative words
neg_df = master_df[master_df['Negative'] == 2009]['Word'].to_list()

In [ ]:
# list of positive words
pos_df  = master_df[master_df['Positive'] == 2009]['Word'].to_list()

In [ ]:
# pos score
def pos_scr():
  pos_lst = [1 for t in p if t.upper() in pos_df]
  pos_scr = sum(pos_lst)
  return(pos_scr)
pos_scr()  

5

In [ ]:
# neg score
def neg_scr():
  neg_lst = [-1 for t in p if t.upper() in neg_df]
  neg_scr = sum(neg_lst)* -1
  return(neg_scr)  
neg_scr()

9

In [ ]:
# polar score
def polar_scr():
  polar_score = (pos_scr() - neg_scr()) / ((pos_scr() + neg_scr())  + 0.000001)
  return(polar_score)

polar_scr()

-0.2857142653061239

In [ ]:
#subjectivity score
def sub_scr():
  sub_score = (pos_scr() + neg_scr()) / ((len(p)) + 0.000001)
  return(sub_score)
sub_scr()

0.02707930362267059

# 2. Analysis of Readibility

In [ ]:
#Average Sentence Length / average number of words per sentence

with open('170.txt', 'r') as file:
  lines = file.read()
#Total_words = len(file_contents.split())
  tokenizer = RegexpTokenizer(r'\w+')
  word_tokens = tokenizer.tokenize(lines)

def avg_slen():
      Total_words = len(word_tokens)
      total_sentences =  lines.count('.')
      return(Total_words/total_sentences)
avg_slen()

29.807692307692307

In [ ]:
import spacy
import textstat
from textstat.textstat import textstatistics,legacy_round
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Count of complex words
def comp_cnt():
  com_cnt = textstat.difficult_words(lines)
  return(com_cnt)
comp_cnt()

139

In [ ]:
## % of complex words
def comp_wrds():
  Total_words = len(word_tokens)
  com_wrds = comp_cnt()/Total_words
  return(com_wrds)
comp_wrds()  

0.17935483870967742

In [ ]:
# fog index

def fog_idx():
  fg_ind = 0.4*(avg_slen() + comp_wrds())
  print(fg_ind)
fog_idx()  

11.994818858560794



# Word Count

In [ ]:
# using nltk to exclude stopwords, exclude punctuations.
# then find number of words

def word_cnt():
  stop_words = set(stopwords.words('english'))
  all_words = [w for w in word_tokens if not w.lower() in stop_words]
  
  count_words = len(all_words)
  return(count_words)

word_cnt()

440

# Syllable count per word

In [ ]:
# avg number of syllables per word
def syl_cnt():
  vowels = 'aeiou'
  countVowels = 0
  for words in word_tokens:
    for letter in words:
      if letter in vowels:
        countVowels += 1
    if words.endswith("ed"):
      countVowels -=1
    if words.endswith("es"):
      countVowels -=1
  return(countVowels/len(word_tokens)) 

syl_cnt()

1.8387096774193548

# Personal Pronouns

In [ ]:
def prp_cnt():
  if 'US' in word_tokens:
    word_tokens.remove('US')
  tagged = nltk.pos_tag(word_tokens)

  counts = Counter(tag for word,tag in tagged)
  filt_keys = ['PRP']
  res = [counts[key] for key in filt_keys]
  strings = [str(res) for res in res]
  a_string = "". join(strings)
  an_integer = int(a_string)
  return(an_integer)

prp_cnt()

7

# Avg Word length

In [ ]:
def avg_wrlen():
  letr_cnt = int()
  Total_words = len(word_tokens)
  for words in word_tokens:
      for letters in words:
          letr_cnt +=1
  
  avg_wrlen = letr_cnt/Total_words
  return(avg_wrlen)
avg_wrlen()

5.078709677419355

# Result input to excel sheet


In [ ]:
import openpyxl
from openpyxl import load_workbook
import string

In [ ]:
excel_file = pd.read_excel('output.xlsx')

In [ ]:

wb = load_workbook('output.xlsx')
sheet = wb['Sheet1']


sheet['C171'] = pos_scr()
sheet['D171'] = neg_scr()
sheet['E171'] = polar_scr()
sheet['F171'] = sub_scr()
sheet['G171'] = avg_slen()
sheet['H171'] = comp_wrds() 
sheet['I171'] = fog_idx()
sheet['J171'] = avg_slen()
sheet['K171'] = comp_cnt()
sheet['L171'] = word_cnt()
sheet['M171'] = syl_cnt()
sheet['N171'] = prp_cnt()
sheet['O171'] = avg_wrlen()


11.994818858560794


In [ ]:
wb.save('output.xlsx')